In [6]:
import geopandas as gpd

# Load the GeoJSON file to examine its content and structure
file_path = '实际公交线路.geojson'
geojson_data = gpd.read_file(file_path)
geojson_data.head()


,id,name,type,start_stop,end_stop,stroke,stroke-width,geometry
0,900000153412,102路(曙光商业街--高铁东站),普通公交,曙光商业街,高铁东站,#132EAE,3,"LINESTRING (112.89854 36.30974, 112.89866 36.3..."
1,900000153411,102路(高铁东站--曙光商业街),普通公交,高铁东站,曙光商业街,#1D8561,3,"LINESTRING (113.16430 36.22686, 113.16419 36.2..."
2,900000139724,黎城102路(庄园--印象小区),普通公交,庄园,印象小区,#445DD9,3,"LINESTRING (113.40382 36.49778, 113.40401 36.4..."
3,900000139615,黎城102路(印象小区--庄园),普通公交,印象小区,庄园,#B107D6,3,"LINESTRING (113.40297 36.51979, 113.40289 36.5..."
4,140400010096,10路(长钢--东关枢纽),普通公交,长钢,东关枢纽,#7FFFB7,3,"LINESTRING (113.05168 36.36168, 113.05170 36.3..."


In [10]:
import numpy as np

def gcj02_to_wgs84_fixed(lng, lat):
    """
    Convert GCJ-02 coordinate to WGS-84 coordinate.
    """
    pi = 3.1415926535897932384626  # π
    a = 6378245.0  # 长半轴
    ee = 0.00669342162296594323  # 偏心率平方
    dlat = _transformlat_fixed(lng - 105.0, lat - 35.0, pi)
    dlng = _transformlng_fixed(lng - 105.0, lat - 35.0, pi)
    radlat = lat / 180.0 * pi
    magic = np.sin(radlat)
    magic = 1 - ee * magic * magic
    sqrtmagic = np.sqrt(magic)
    dlat = (dlat * 180.0) / ((a * (1 - ee)) / (magic * sqrtmagic) * pi)
    dlng = (dlng * 180.0) / (a / sqrtmagic * np.cos(radlat) * pi)
    mglat = lat + dlat
    mglng = lng + dlng
    return lng * 2 - mglng, lat * 2 - mglat

def _transformlat_fixed(lng, lat, pi):
    ret = -100.0 + 2.0 * lng + 3.0 * lat + 0.2 * lat * lat + 0.1 * lng * lat + 0.2 * np.sqrt(np.abs(lng))
    ret += (20.0 * np.sin(6.0 * lng * pi) + 20.0 * np.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * np.sin(lat * pi) + 40.0 * np.sin(lat / 3.0 * pi)) * 2.0 / 3.0
    ret += (160.0 * np.sin(lat / 12.0 * pi) + 320 * np.sin(lat * pi / 30.0)) * 2.0 / 3.0
    return ret

def _transformlng_fixed(lng, lat, pi):
    ret = 300.0 + lng + 2.0 * lat + 0.1 * lng * lng + 0.1 * lng * lat + 0.1 * np.sqrt(np.abs(lng))
    ret += (20.0 * np.sin(6.0 * lng * pi) + 20.0 * np.sin(2.0 * lng * pi)) * 2.0 / 3.0
    ret += (20.0 * np.sin(lng * pi) + 40.0 * np.sin(lng / 3.0 * pi)) * 2.0 / 3.0
    ret += (150.0 * np.sin(lng / 12.0 * pi) + 300.0 * np.sin(lng / 30.0 * pi)) * 2.0 / 3.0
    return ret

# Apply transformation to all coordinates in the GeoDataFrame
geojson_data['geometry'] = geojson_data['geometry'].apply(
    lambda line: np.array([gcj02_to_wgs84_fixed(x, y) for x, y in np.array(line.coords)])
)

# Check if the coordinates have been transformed correctly
geojson_data.head()



C:\Users\zhong'shuai\AppData\Local\Temp\ipykernel_31092\295762276.py:37: UserWarning: Geometry column does not contain geometry.
  geojson_data['geometry'] = geojson_data['geometry'].apply(


,id,name,type,start_stop,end_stop,stroke,stroke-width,geometry
0,900000153412,102路(曙光商业街--高铁东站),普通公交,曙光商业街,高铁东站,#132EAE,3,"[[112.89238607066287, 36.30990337927841], [112..."
1,900000153411,102路(高铁东站--曙光商业街),普通公交,高铁东站,曙光商业街,#1D8561,3,"[[113.15770084518614, 36.22675368268582], [113..."
2,900000139724,黎城102路(庄园--印象小区),普通公交,庄园,印象小区,#445DD9,3,"[[113.39707930160283, 36.49749388826514], [113..."
3,900000139615,黎城102路(印象小区--庄园),普通公交,印象小区,庄园,#B107D6,3,"[[113.39622654420562, 36.519492308465274], [11..."
4,140400010096,10路(长钢--东关枢纽),普通公交,长钢,东关枢纽,#7FFFB7,3,"[[113.045060263852, 36.3615047066522], [113.04..."
